In [ ]:
from IPython.display import Javascript

Javascript('''
(async () => {
  let div = document.createElement('div');
  div.id = 'p5-container';
  document.body.appendChild(div);

  if (!window.p5) {
    let s = document.createElement('script');
    s.src = "https://cdnjs.cloudflare.com/ajax/libs/p5.js/1.9.0/p5.js";
    document.head.appendChild(s);
    await new Promise(r => s.onload = r);
  }

  new p5((p) => {
    let bolaX, bolaY;
    let vx, vy;
    let v0 = 20;
    let angle = 45;
    let shooting = false;

    // Tempo e escala
    const dt = 0.05; // passo de tempo pequeno para integração mais precisa
    const scale = 10; // pixels por metro (escala da simulação)

    // Física realista da bola
    const rho = 1.225;   // densidade do ar (kg/m³)
    const Cd = 0.47;     // coeficiente de arrasto (esfera)
    const r = 0.0366;    // raio em metros (~bola de beisebol)
    const A = Math.PI * r * r; // área frontal
    const m = 0.145;     // massa em kg (bola de beisebol)

    const g = 9.8;       // gravidade m/s²

    const canoComprimento = 80;
    const canoAltura = 20;
    const baseX = 120;
    const baseY = 380;

    const alvoX = 850;
    const alvoY = 350;
    const alvoRaio = 50;

    let angleSlider, speedSlider;
    let fireButton;
    let hudDiv;

    let showMessage = false;
    let messageTimer = 0;

    p.setup = function() {
      let canvas = p.createCanvas(900, 450);
      canvas.parent(div);
      p.imageMode(p.CENTER);

      hudDiv = p.createDiv("Ângulo: " + angle + "° Velocidade: " + v0 + " m/s");
      hudDiv.position(p.width + 20, 20);
      hudDiv.style('background-color', 'rgba(0,0,0,0.7)');
      hudDiv.style('color', 'white');
      hudDiv.style('padding', '12px 20px');
      hudDiv.style('border-radius', '8px');
      hudDiv.style('font-size', '18px');
      hudDiv.style('font-weight', 'bold');
      hudDiv.style('line-height', '1.5em');

      angleSlider = p.createSlider(10, 80, 45);
      angleSlider.position(p.width + 20, 100);
      angleSlider.style('width', '180px');

      speedSlider = p.createSlider(5, 50, 20); // até 50 m/s
      speedSlider.position(p.width + 20, 150);
      speedSlider.style('width', '180px');

      fireButton = p.createButton("Disparar");
      fireButton.position(p.width + 20, 210);
      fireButton.style('font-size', '16px');
      fireButton.style('padding', '5px 15px');
      fireButton.mousePressed(() => {
        shooting = true;
        let rad = p.radians(angleSlider.value());
        bolaX = baseX + canoComprimento * p.cos(rad);
        bolaY = baseY - canoComprimento * p.sin(rad);
        vx = speedSlider.value() * Math.cos(rad);
        vy = -speedSlider.value() * Math.sin(rad);
      });
    };

    p.draw = function() {
      p.background(135, 206, 235);

      // chão
      p.fill(34, 139, 34);
      p.rect(0, 400, p.width, 50);

      angle = angleSlider.value();
      v0 = speedSlider.value();
      let rad = p.radians(angle);

      hudDiv.html("Ângulo: " + angle + "° Velocidade: " + v0 + " m/s");

      // pré-visualização da trajetória
      let previewX = baseX + canoComprimento * p.cos(rad);
      let previewY = baseY - canoComprimento * p.sin(rad);
      let pvx = v0 * Math.cos(rad);
      let pvy = -v0 * Math.sin(rad);

      p.stroke(0);
      p.strokeWeight(2);
      for (let step = 0; step < 500; step++) {
        let v = Math.sqrt(pvx*pvx + pvy*pvy);

        // ARRASO REALISTA
        let Fd = 0.5 * rho * Cd * A * v * v; // força de arrasto
        let ax = -Fd * (pvx / v) / m; // aceleração X
        let ay = -Fd * (pvy / v) / m + g; // aceleração Y (gravidade positiva para baixo)

        pvx += ax * dt;
        pvy += ay * dt;

        previewX += pvx * dt * scale;
        previewY += pvy * dt * scale;

        if (previewY >= 400) break;
        p.point(previewX, previewY);
      }
      p.noStroke();

      // movimento real da bola
      if (shooting) {
        let v = Math.sqrt(vx*vx + vy*vy);
        let Fd = 0.5 * rho * Cd * A * v * v; // força de arrasto
        let ax = -Fd * (vx / v) / m;
        let ay = -Fd * (vy / v) / m + g;

        vx += ax * dt;
        vy += ay * dt;

        bolaX += vx * dt * scale;
        bolaY += vy * dt * scale;

        p.fill(0);
        p.circle(bolaX, bolaY, 30);

        let d = p.dist(bolaX, bolaY, alvoX, alvoY);
        if (d < alvoRaio) {
          shooting = false;
          showMessage = true;
          messageTimer = p.millis();
        }

        if (bolaY >= 400 - 15) shooting = false;
      }

      // Canhão
      p.push();
      p.translate(baseX, baseY);
      p.rotate(-rad);
      p.fill(80);
      p.rect(0, -canoAltura/2, canoComprimento, canoAltura, 5);
      p.pop();

      // Base do canhão
      p.push();
      p.translate(baseX, baseY);
      p.fill(100);
      p.ellipse(0, 10, 50, 20);
      p.pop();

      // alvo
      const cores = ['white', 'black', 'blue', 'red', 'yellow'];
      for (let i = 0; i < 5; i++) {
        p.fill(cores[i]);
        p.stroke(0);
        p.strokeWeight(2);
        p.ellipse(alvoX, alvoY, alvoRaio*2 - i*15, alvoRaio*2 - i*15);
      }

      // mensagem
      if (showMessage) {
        p.textSize(32);
        p.fill('yellow');
        p.stroke(0);
        p.strokeWeight(3);
        p.textAlign(p.CENTER, p.CENTER);
        p.text("Parabéns!", p.width/2, p.height/2);

        if (p.millis() - messageTimer > 1000) showMessage = false;
      }
    };
  });
})();
''')


<IPython.core.display.Javascript object>